# Coletando dados de uma API

# Visão geral
Esse caderno tem como objetivo coletar os dados da API publica CIVIC (Clinical Interpretation of variants in cancer) e transforma-los em dataframes.
Nossa API possui 4 diferentes URL's das quais teremos que receber os dados, ou seja, repitiremos o mesmo processo para todas as URL's.
## Objetivo do código
Acessar a API CIVIC e coletar os dados fornecidos por ela.

In [1]:
#Importações
import requests
import json
import pandas as pd

Inicialmente, iremos realizar as importações necessárias. Precisaremos da biblioteca requests, que nos permite fazer solicitações em HTTP com os métodos padrões. Usaremos o método GET para todas as URLs.

A biblioteca Json fornece uma maneira fácil de codificar e decodificar dados no formato JSON. Esse formato é comumente utilizado para trocar informações entre diferentes sistemas, sendo amplamente utilizado em serviços web e APIs RESTful.

Por último, usaremos a biblioteca Pandas para análise e manipulação de dados. Essa biblioteca fornece estruturas de dados de alta performance.

Para cada uma das APIs, faremos uma solicitação GET, sendo que cada uma delas possui um retorno diferente. A fim de garantir que a solicitação de coleta tenha sido aceita, verificaremos o "status_code". Os códigos mais comuns incluem:

200: tudo funcionou conforme o esperado e a validação dos dados foi realizada com sucesso.

400: ocorreram problemas durante a autenticação.

401: este erro ocorre quando há algum problema na requisição.

500: ocorreu algum erro interno no servidor.

Além disso, manteremos um padrão de 10.000 para o retorno.

# Assertion
https://civic.readthedocs.io/en/latest/api/v1/assertions.html

Assertions, que possui um total de 48 valores de retorno divididos em 48 páginas. Para realizar a coleta desses valores, definiremos a URL informada pelo site "https://civicdb.org/api/assertions" e mudaremos o parâmetro "count" de 1 (valor padrão). Dessa forma, receberemos todos os dados em uma única chamada.

# Evidences
https://civic.readthedocs.io/en/latest/api/v1/evidence.html

Evidences, que possui um total de 8939 valores de retorno divididos em 8939 páginas. Para realizar a coleta desses valores, definiremos a URL informada pelo site "https://civicdb.org/api/evidence_items".

# Variant
https://civic.readthedocs.io/en/latest/api/v1/variants.html

Variant, que possui um total de 3163 valores de retorno divididos em 3163 páginas. Para realizar a coleta desses valores, definiremos a URL informada pelo site "https://civicdb.org/api/variants".

# Gene
https://civic.readthedocs.io/en/latest/api/v1/genes.html

Gene, que possui um total de 332 valores de retorno divididos em 332 páginas. Para realizar a coleta desses valores, definiremos a URL informada pelo site "https://civicdb.org/api/genes".

In [2]:
#URL da API
Assertion = requests.get('https://civicdb.org/api/assertions?count=10000')
Evidences = requests.get('https://civicdb.org/api/evidence_items?count=10000')
Variant = requests.get('https://civicdb.org/api/variants?count=10000')
Gene = requests.get('https://civicdb.org/api/genes?count=10000')
#Codigo de status 

In [3]:
#Codigo de status 
print(f"Status_code de Assertion: {Assertion.status_code}")
print(f"Status_code de Evidences: {Evidences.status_code}")
print(f"Status_code de Variant: {Variant.status_code}")
print(f"Status_code de Gene: {Gene.status_code}")

Status_code de Assertion: 200
Status_code de Evidences: 200
Status_code de Variant: 200
Status_code de Gene: 200


Json.loads() É um método usado para decodificar uma string JSON em um objeto Python. O objeto Python resultante pode ser uma lista, um dicionário, um valor de string, um número, um valor booleano ou None. 
A função .content é usada para retornar o conteúdo de uma resposta HTTP em formato de bytes.

Converteremos o conteúdo da resposta HTTP, que é originalmente uma string no formato JSON, para um dicionário Python utilizando a função .loads.

In [4]:
#Json.load: Pega um objeto em arquivo e retorna o objeto json;
#contente: Retorna o contéudo da resposta em dicionario.
Assertion_dict=json.loads(Assertion.content)
Evidences_dict=json.loads(Evidences.content)
Variant_dict=json.loads(Variant.content)
Gene_dict=json.loads(Gene.content)

Utilizando o método pd.DataFrame iremos converter o dicionário Python em um dataframe pandas com os dados retornados pela API.

In [5]:
#Pega apenas uma das chaves que a URL retorna e a transforma em um dataframe.
df_assertion=pd.DataFrame(Assertion_dict["records"])
df_evidences=pd.DataFrame(Evidences_dict["records"])
df_variant=pd.DataFrame(Variant_dict["records"])
df_gene=pd.DataFrame(Gene_dict["records"])

In [6]:
#Dataframe
#df_assertion

In [7]:
#Dataframe
#df_evidences

In [8]:
#Dataframe
#df_variant 

In [9]:
#Dataframe
#df_gene 

Por ultimo nessa URL iremos salvar o Dataframe criado para que ele possa ser usado para posterior análise e manipulação.

In [10]:
#Salva o dataframe criado.
df_assertion.to_csv('Assertions.csv', index=False)
df_evidences.to_csv('Evidence.csv', index=False)
df_variant.to_csv('Variants.csv', index=False)
df_gene.to_csv('Genes.csv', index=False)

Em virtude das informações citadas acima, entendemos que esse código busca um grande conjunto de dados em 4 URL's diferentes, Gene, Evidences, Variants, Assertions e os transforma em dataset para que possam ser usados posteriormente.

# Junção CSV's Civic

Infelizmente, os dados que a API oferece não são suficientes para a realização da nossa análise. Por isso, usaremos dados fornecidos pelo CIVIC em formato TSV, que precisam ser convertidos para CSVs para que possamos usa-los. Assim como na API, os arquivos TSV também estão separados e precisam ser mesclados. Para isso, iremos utilizar a função "merge", que é usada para combinar dois ou mais conjuntos de dados com base em uma chave comum. Para essa etapa, precisamos apenas da importação do Pandas.

Abaixo, definiremos os arquivos que iremos utilizar.

In [11]:
#Definiçao dos arquivos que serão utilizados
df_Variant = pd.read_csv('01-Apr-2022-VariantSummaries.csv')
df_Evidences = pd.read_csv('01-Apr-2022-ClinicalEvidenceSummaries.csv')
df_Gene = pd.read_csv('01-Apr-2022-GeneSummaries.csv')

Possuímos três DataFrames diferentes: df_Variant, que possui **1533** linhas e **33** colunas, df_evidences, que possui **3897** linhas e **42** colunas e df_gene com **477** linhas e **7** colunas.

Antes de podermos realizar o merge, precisamos fazer um pequeno tratamento nos dados fornecidos. Inicialmente, precisamos trocar o nome de uma coluna do df_Variant, que possuem o mesmo valor e significado no df_evidences, porém com nome diferente. Para isso, usaremos o método "rename".

In [12]:
df_Variant.rename(columns={'summary': 'variant_summary'}, inplace = True)

Após trocarmos o nomes dessa coluna, precisamos apagar outras do mesmo df, pois essas colunas não se encontram em nenhum outro df, e por conta disso, a mesclagem se torna um problema. 
Por isso, essas colunas estão sendo excluídas, mas nenhum dado importante será perdido.

In [13]:
df_Variant = df_Variant.drop(['assertion_ids', 'is_flagged'],axis=1)

Com o df tratado, podemos realizar o primeiro merge. Inicialmente, iremos juntar o df_Variant tratado e o df_Evidences, ambos com a maior quantidade de linhas e colunas.
Assim, criaremos um df nomeado Civic que possui a junção de todos os registros dos dois conjuntos de dados originais.

Para que essa mesclagem funcione, usaremos como chave a coluna 'variant_id' e como parâmetro usaremos o left_on e o right_on, fazendo o uso desses parâmetros teremos especificado as colunas que serão utilizadas como chaves de junção para o conjunto de dados.

Para que nossos dados não fiquem poluídos e que não precisemos ter um trabalho extra excluindo colunas repetidas que serão geradas com o merge, pois nosso conjunto de dados possuem colunas com o mesmo nome, iremos definir o parâmetro "suffixes" como uma tupla vazia e uma tupla com o sufixo "_y", o que garantirá que não haverá colunas duplicadas após a junção.

Após a junção, usaremos o método "filter" para selecionar apenas as colunas que possuem o sufixo "_y" no nome e usaremos o método "drop" para removê-las.

In [14]:
#Merge entre Civic e df_Variant
Civic = pd.merge(df_Evidences, df_Variant,  left_on='variant_id', right_on='variant_id', how='left', suffixes=('','_y'))
Civic.drop(Civic.filter(regex='_y$').columns, axis=1, inplace=True)

Com dois DataFrames iniciais juntos, seguiremos para o df_genes. Assim como o df_Variant, ele precisa ser tratado antes da junção, porém por motivos diferentes. O df_genes possui valores duplicados que se encontram em outros DataFrames. Esses valores serão excluídos, e manteremos apenas as colunas "description", que não possuímos em outros DataFrames, e "name", que usaremos como chave de junção.

In [15]:
#Apagando colunas
df_Gene = df_Gene.drop(['gene_civic_url', 'last_review_date', 'is_flagged'],axis=1)

Também não possuímos a coluna "name" em nenhum outro dataframe, mas seus valores correspondem às colunas denominadas como "gene" em outros, por isso faremos o metodo rename.

In [16]:
#Trocando o nome da coluna name para gene.
df_Gene.rename(columns={'name': 'gene'}, inplace = True)

Com o "df_gene" tratado, realizaremos o último merge juntando-o ao conjunto total. Usaremos as colunas "gene" como chave de junção e passaremos o mesmo parâmetro "suffixes" para apagar colunas duplicadas.

In [17]:
#Merge entre Civic e df_gene
Civic = pd.merge(Civic, df_Gene,  left_on='gene', right_on='gene', how='left', suffixes=('','_y'))
Civic.drop(Civic.filter(regex='_y$').columns, axis=1, inplace=True)

Agora, com o dataframe pronto, realizaremos o último tratamento dessa etapa. Aqui, apagaremos colunas sem valor para nossa análise, como URLs e a última edição ("last_review_date"), colunas com valores nulos em todos os dataframes e colunas que não serão usadas.

In [18]:
#Removendo colunas
Civic = Civic.drop(['gene_civic_url', 'Unnamed: 32', 'Unnamed: 31', 'Unnamed: 29',
                    'Unnamed: 30', 'assertion_civic_urls', 'variant_civic_url', 
                    'evidence_civic_url', 'last_review_date', 'asco_abstract_id',
                    'nct_ids', 'chromosome2', 'start2', 'stop2', 'representative_transcript2',
                    'variant_groups'
],axis=1)
Civic

,gene,entrez_id,variant,disease,doid,phenotypes,drugs,drug_interaction_type,evidence_type,evidence_direction,...,variant_summary,variant_origin,is_flagged,variant_types,hgvs_expressions,civic_variant_evidence_score,allele_registry_id,clinvar_ids,variant_aliases,description
0,JAK2,3717,V617F,Lymphoid Leukemia,1037.0,NaN,NaN,NaN,Diagnostic,Supports,...,JAK2 V617F is a highly recurrent mutation in m...,Somatic,False,"missense_variant,gain_of_function_variant","NC_000009.11:g.5073770G>T,NM_004972.3:c.1849G>...",83.0,CA124183,14662,"VAL617PHE,RS77375493",JAK2 is a kinase that is misregulated or mutat...
1,PDGFRA,5156,D842V,Gastrointestinal Stromal Tumor,9253.0,NaN,NaN,NaN,Diagnostic,Supports,...,PDGFRA D842 mutations are characterized broadl...,Somatic,False,missense_variant,"NM_006206.4:c.2525A>T,NP_006197.1:p.Asp842Val,...",100.5,CA123194,13543,"ASP842VAL,RS121908585","Commonly mutated in GI tract tumors, PDGFR fam..."
2,DNMT3A,1788,R882,Acute Myeloid Leukemia,9119.0,NaN,NaN,NaN,Diagnostic,Supports,...,DNMT3A R882 mutations are associated with cyto...,Somatic,False,missense_variant,NaN,510.0,NaN,"375882,375883,375884,375879,375880,375881","R693,ARG882,ARG693",DNMT3A is one of several epigenetic modifiers ...
3,DNMT3A,1788,R882,Acute Myeloid Leukemia,9119.0,NaN,NaN,NaN,Diagnostic,Supports,...,DNMT3A R882 mutations are associated with cyto...,Somatic,False,missense_variant,NaN,510.0,NaN,"375882,375883,375884,375879,375880,375881","R693,ARG882,ARG693",DNMT3A is one of several epigenetic modifiers ...
4,JAK2,3717,V617F,Chronic Myeloid Leukemia,8552.0,NaN,NaN,NaN,Diagnostic,Supports,...,JAK2 V617F is a highly recurrent mutation in m...,Somatic,False,"missense_variant,gain_of_function_variant","NC_000009.11:g.5073770G>T,NM_004972.3:c.1849G>...",83.0,CA124183,14662,"VAL617PHE,RS77375493",JAK2 is a kinase that is misregulated or mutat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3892,ERCC1,2067,Underexpression,Ovary Epithelial Cancer,2152.0,NaN,Platinum Compound,NaN,Predictive,Supports,...,NaN,NaN,False,NaN,NaN,15.0,NaN,NaN,NaN,NaN
3893,NTRK1,4914,Amplification,Solid Tumor,NaN,NaN,Larotrectinib,NaN,Predictive,Supports,...,NaN,Somatic,False,transcript_amplification,NaN,5.0,NaN,NaN,NaN,NaN
3894,CTCF,10664,K365T,NaN,NaN,NaN,NaN,NaN,Functional,Supports,...,NaN,NaN,False,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3895,KIT,3815,EXPRESSION,Glioblastoma,3068.0,NaN,Sunitinib,NaN,Predictive,Supports,...,NaN,Somatic,False,NaN,NaN,20.0,NaN,NaN,NaN,c-KIT activation has been shown to have oncoge...


Para que o dataframe "Civic" final possa ser utilizado pelo restante do grupo, iremos salvá-lo como CSV.

In [19]:
Civic.columns

Index(['gene', 'entrez_id', 'variant', 'disease', 'doid', 'phenotypes',
       'drugs', 'drug_interaction_type', 'evidence_type', 'evidence_direction',
       'evidence_level', 'clinical_significance', 'evidence_statement',
       'citation_id', 'source_type', 'citation', 'rating', 'evidence_status',
       'evidence_id', 'variant_id', 'gene_id', 'chromosome', 'start', 'stop',
       'reference_bases', 'variant_bases', 'representative_transcript',
       'ensembl_version', 'reference_build', 'variant_summary',
       'variant_origin', 'is_flagged', 'variant_types', 'hgvs_expressions',
       'civic_variant_evidence_score', 'allele_registry_id', 'clinvar_ids',
       'variant_aliases', 'description'],
      dtype='object')

In [20]:
#Salvando o df Civic como um csv
Civic.to_csv('Civic.csv', index=False)

Diante disso, podemos ver que nosso código consiste na leitura de quatro arquivos csv (df_Variant, df_Evidence, df_Gene, Df_Assertion) fornecidos pelo proprio Civic, e na sua mesclagem em um dataframe final chamado Civic, tambem possuimos alguns tratamentos sendo realizados nos dados, como a exclusão de colunas não relevantes, a renomeação de colunas e a remoçao de valores duplicados.
Ao final o Dataframe Civic é salvo em um arquivo csv para ser utilizado posteriormente.

# Análise Descritiva.

## df_Variant
Total de linhas **1533**, total de colunas **33**


Visão geral:
As variantes CIViC são geralmente alterações genômicas, incluindo variantes de nucleotídeo único (SNVs), eventos de inserção/deleção (indels), alterações no número de cópias (CNVs; como amplificações ou deleções), variantes estruturais (SVs; como translocações ou inversões) e outras eventos que diferem do genoma “normal”. Em alguns casos, uma variante CIViC pode representar eventos do transcriptoma ou proteoma. Por exemplo, 'expression' ou 'over-expression' é uma variante válida. Novos curadores geralmente devem evitar propor novas variantes que sejam diferentes de qualquer outra já existente no CIViC. Todas as Variantes CIViC estão associadas a um ou mais Perfis Moleculares com alguma evidência de relevância clínica (preditiva, prognóstica, diagnóstica, etc).

- Variant_id: Id que define a Variant (Index)
- Variant_civic_url: Urls onde informações extras sobre as Variants se encontram.
- Gene: Nome do gene associado a Variant.
- Entrez_id: Id do gene.
- Variant: Nome da variant.
- Summary: Descrição e associação da variant.
- Varian_group: Em qual grupo a variant se encontra.
- Chromosome: Nome do cromossomo no qual ocorre a variants.
- Start: Posição inicial da variant (cordenada do genoma baseado em 1).
- Stop: Posição final da variant (deve ser igual ou menor a posição inicial).
- Reference_base: A base nucleotídica do alelo de referência e variante.
- Variant_base: A base nucleotídica do alelo da variante.
- Representative_transcript: ID da transcrição do conjunto e número da versão para uma transcrição conhecida do gene que contém a variant.
- Ensembl_version: Versão de compilação da anotação Ensemble.
- Reference_build: Versão de montagem de referência humana.
- Chromosome2: Nome do segundo cromossomo no qual ocorre a variants.
- Start2: Segunda posição inicial variant (cordenada do genoma baseado em 1). 
- Stop2: Segunda posição final da variant (deve ser igual ou menor a posição inicial).
- Representative_transcript: ID da transcrição do conjunto e número da versão para uma transcrição conhecida do gene que contém a variant.
- Variant_type: Descreve o tipo de variant.
- hgvs_expressions: Representação da variant no nível de DNA, RNA ou proteína.
- last_review_data: Ultima data de atualização.
- civic_variant_evidence_score: Pontuação do perfil molecular.
- Allele_registry_id: ID do allelo. (Allelos são alternativas, geradas pelas mutações de um mesmo gene).
- Clinvar_ids: ID Clivar
- Variant_aliases: Nomes alternativos para as variants.
- Assertion_ids: ID de asserções.
- Assertion_civic_urls: Asserções urls.
- Is flagged: Fácil detecção. (Se a variant foi detectada ou não antes dos experimentos.)

### Valores nulos dentro de Variant.
- summary                         1283
- variant_groups                  1411
- chromosome, start e stop         300
- reference_bases                  796
- variant_bases                    837
- representative_transcript        324
- ensembl_version                  332
- reference_build                  292
- chromosome2, start2, stop2 e representative_transcript2    1482
- variant_types                    398
- hgvs_expressions                 781
- allele_registry_id               764
- clinvar_ids                      766
- variant_aliases                  644
- assertion_ids e assertion_civic_urls                       1502
- Unnamed: 29                     1527
- Unnamed: 30                     1531
- Unnamed: 31 e Unnamed: 32       1532

### Valores duplicados dentro de Variant.
- Gene e entrez_id 1201
- variant 349
- summary 1293
- variant_groups 1505
- chromosome 1509
- start 549
- stop 544
- reference_bases 1478
- variant_bases 1490 
- representative_transcript 1199
- reference_build 1530
- chromosome2 1513
- start2 1499
- stop 1498
- variant_types 1437
- hgvs_expressions 783
- last_review_date 426
- civic_variant_evidence_score 1357
- allele_registry_id 781
- clinvar_ids 1053
- variant_aliases 645
- assertion_ids e assertion_civic_urls 1501
- is_flagged 1525
- Unnamed: 29 1529
- Unnamed: 30 1530
- Unnamed: 31 e Unnamed: 32 1531

# df_Evidences
Total de linhas **3897** , total de colunas **42**
Visão geral: A declaração de evidência clínica é um pedaço de informação que foi selecionada manualmente da literatura médica confiável sobre um perfil molecular (variante) ou 'evento' genômico que tem implicações para a função da proteína, oncogenicidade, predisposição ao câncer, diagnóstico (também conhecido como classificação molecular), prognóstico , ou resposta preditiva à terapia. Por exemplo, “Pacientes com mutações BRAF V600 respondem bem ao medicamento dabrafenibe”. Um perfil molecular é composto por uma ou mais variantes que podem ser uma única substituição de nucleotídeo, uma pequena inserção ou deleção, uma fusão de genes de RNA, um rearranjo cromossômico, um padrão de expressão de RNA (por exemplo, superexpressão), etc. declaração de evidência clínica corresponde a uma única publicação citável.

- Gene: Nome do gene;
- entrez_id: ID do gene;
- variant: Nome da variante;
- disease: Doença específica ou subtípo dedoença associada ao evento e sua implicação clínica;
- doid: ID de disease;
- Phenotypes: Fenótipo específicos associados a declaração de evidências;
- drugs: Medicamento/Droga usada;
- drug_interaction_type: Se a medicação usada foi alterada, subistituida ou acresentada. 
- evidence_type: Tipos de diagnósticos possiveis.
- evidence_direction: Se o paciente suporta o medicamento.
- evidence_level: Descreve a robustez do tipo de estudo que suporta a evidencia. ("A- Associação valida", "B-Evidência clínica", "C-Estudo de caso", "D-Evidência pré-clínica" e "E-Associação inferencial");
- clinical_significance: Resposta clinica ao tratamento realizado;
- evidence_statement: Status da evidência após o tratamento;
- citation_id: ID da citação clínica;
- source_type: Onde o evento esta descrito;
- asco_abstract_id: ID de source_type;
- citation: Local e data onde a citação foi publicada;
- nct_ids: ID do ensaio clínico associado a evidência;
- rating: Nota dada para o estudo realizado;
- evidence_status: Status de aceitação para o tratamento;
- evidence_id: ID de evidence;
- variant_id: ID de variavel;
- gene_id: ID de gene;
- variant_summary: Descrição e associação da variant;
- variant_origin: Como a variant começou;
- last_review_date: Data da ultima atualização feita;
- evidence_civic_url: Link para evidence;
- variant_civic_url: Link para variant;
- gene_civic_url: Link para Gene;
- is_flagged: Fácil detecção.

## Valores nulos dentro de evidences.

- disease                         23
- doid                            32
- phenotypes                    3389
- drugs                         1448
- drug_interaction_type         3278
- evidence_direction             179
- clinical_significance          222
- asco_abstract_id              3885
- nct_ids                       3401
- rating                          68
- chromosome                     435
- start e stop                   436
- reference_bases               2000
- variant_bases                 2055
- representative_transcript      466
- chromosome2, start2, stop2 e representative_transcript2          3748
- ensembl_version                475
- reference_build                426
- variant_summary               2510
- variant_origin                 348

### Valores duplicados dentro de Variant.
- Gene e entrez_id 3565
- variant 2713
- disease 3659
- doid 3663
- phenotypes 3734
- drugs 3289
- drug_interaction_type 3893
- evidence_type  3891
- evidence_direction 3894
- evidence_level 3892
- clinical_significance 3881
- evidence_statement 222
- citation_id 2192
- source_type 3895
- asco_abstract_id 3885
- citation 2202
- nct_ids 3719
- rating 3891
- evidence_status 3896
- variant_id 2364
- gene_id 3565
- chromosome 3873
- start 2913
- stop 2908
- reference_bases 3842
- variant_bases 3854
- representative_transcript 3563
- chromosome2 3877
- start2 3863
- start2 3863
- representative_transcript2 3866 
- ensembl_version 3893
- reference_build 3894
- variant_summary 3657
- variant_origin 3892
- last_review_date 1107
- variant_civic_url 2364
- gene_civic_url  3565
- is_flagged 3895

# df_Gene
Total de linhas **477**, total de colunas **7**
Visão Geral: A entidade do gene no banco de dados CIViC inclui vários recursos úteis para avaliar a relevância clínica das variantes. O primeiro é um resumo em nível de gene com curadoria humana que descreve a relevância clínica do gene com fontes associadas.

- gene_id: Index do gene; 
- gene_civic_url: Link para gene;
- name: nome do gene;
- entrez_id: ID do gene;
- description: Descrição do gene e suas caracteristicas 
- last_review_data: Ultima atualização;
- is_flagged = Fácil detecção.

## Valores nulos em gene
- description         415

## Valores duplicados em gene
- description 415 
- last_review_date 95 
- is_flagged 476

# Civic
Total de linhas **3897**, total de colunas **39**.

- Gene: Nome do gene que sofreu a multação;
- entrez_id: ID do gene;
- variant: Descrição do tipo de variante;
- disease: Doença específica ou subtipo de doença associada;
- doid: ID de disease;
- phenotypes: Fenótipo específico associado a declaração de evidências;
- drugs: Droga usada durante o tratamento;
- evidence_type: Categoria da ação clínica/relevância implicada pelo evento. (Predictive - Evidência pertencente ao efeito de um perfil molecular na resposta terapêutica. Diagnostic - Evidência relativa ao impacto de um perfil molecular no diagnóstico do paciente. Prognostic - Evidência pertencente ao impacto de um perfil molecular na progressão da doença, gravidade ou sobrevida do paciente; Predisposing - A evidência refere-se ao papel de um perfil molecular da linhagem germinativa em conferir suscetibilidade à doença (incluindo avaliações de patogenicidade); Oncogenic - A evidência refere-se ao envolvimento de um perfil molecular somático na patogênese do tumor; Functional - A evidência refere-se ao impacto de um perfil molecular na função do produto gênico(por exemplo, proteína));
- evidence_direction: Um indicador de se a declaração de evidência apóia ou refuta a importância de um evento. (Supports ou Does not support);
- clinical_significance: O impacto do perfil molecular (uma ou mais variantes) para tipos de evidências preditivas, prognósticas, diagnósticas, oncogênicas ou funcionais. (Sensitivity/Response - Associada a uma resposta clínica ou pré-clínica ao tratamento; Reduced Sensitivity - A resposta ao tratamento é menor do que a observada em outros contextos de tratamento; Resistance - Associada á resistência clínica ou pré-clínica ao tratamento; Adverse Response - Associada a uma resposta adversa ao tratamento medicamentoso; N/A - A variante não informa o interpretação clínica; Positive - Associada ao diagnóstico de doença ou subtipo; Negative - Associada á ausência de doença ou subtipo; Better outcome - Demonstra resultado clínico melhor do que o esperado; Poor outcome - Demonstra resultado clínico pior do que o esperado; Pathogenic - As evidências podem contribuir para uma interpretação de patogênico/benigno no nível de afirmação; Proteção - A presença de um perfil molecular germinativo reduz a chance de cancêr;
- variant_id: ID da variante.
- gene_id: ID do gene (Exibido automaticamente pela interface CIVIC.)
- is_flagged: Se o paciente deu algum sinal de que possuia cancêr antes do inicio do tratamento. (Fácil detecção);
- drug_interaction_type: Se a droga usada durante o tratamento teve que ser substituida ou combinada/complementada;
- evidence_level: Descreve a robustez do tipo de estudo que suporta o item de evidência; (A - Associação validad, B - Evidência clínica, C - Estudo de caso, D - Evidência pré-clínica, E - Associação inferencial);
- evidence_statement: Detalhes de tipos específicos de declarações de evidências: Uma reiteração do tipo de evidência, perfil molecular (variante), gene e doença;
- citation_id: ID de citation
- source_type: Tipo de fonte e ID, vincula o item de evidência à publicação original que suporta as reivindicações de evidência.
- citation: Publicação onde o evento foi descrito/explorado gerado automaticamente a partir do ID PudMed fornecido pelo curador e links para a página interna da publicaçãi CIVIC mostrando todos os itens de evidência da publicação;
- rating: Discernir rapidamente o quão bem a evidência derivada do artigo ou fonte suporta uma determinada declaração de evidência, um sistema de classificação de cinco estrelas é usado, (5 -Evidência fortes e bem fundamentadas de um laboratório ou periódico com reputação acadêmica respeitada. Os experimentos são bem controlados e os resultados são limpos e reprodutíveis em várias réplicas. Evidências confirmadas por métodos independentes. O estudo é estatisticamente bem alimentado, 4 - Evidência forte e bem suportada. As experiências são bem controladas e os resultados são convincentes. Quaisquer discrepâncias dos resultados esperados são bem explicadas e não são preocupantes, 3 - As evidências são convincentes, mas não apoiadas por uma variedade de experimentos. Podem ser projetos de menor escala ou novos resultados sem muitos experimentos de acompanhamento. As discrepâncias dos resultados esperados são explicadas e não são preocupantes, 2 - As evidências não são bem apoiadas por dados experimentais e poucos dados de acompanhamento estão disponíveis. A publicação é de um periódico de baixo impacto acadêmico. Os experimentos podem carecer de controles adequados, ter tamanho de amostra pequeno ou não serem estatisticamente convincentes, 1 - A alegação não é bem apoiada por evidências experimentais. Os resultados não são reprodutíveis ou têm um tamanho de amostra muito pequeno. Nenhum acompanhamento é feito para validar novas reivindicações.);
- evidence_status: Se o experimento apoio ou não apoia a variante usada;
- evidence_id: ID da evidencia;
- chromosome: Nome do cromossomo no qual ocorre a variante;
- stat: Posições inicial da variante (coordenada do genoma baseada em 1).
- stop: Posições final da variante. A partida deve ser menor ou igual à parada.
- reference_bases: A base nucleotídica do alelo de referência e variante (por exemplo, 'C', 'A');
- variant_bases: A base nucleotídica do alelo de referência e variante (por exemplo, 'C', 'A');
- representative_transcript: ID da transcrição do conjunto e número da versão para uma transcrição conhecida do gene que contém a variante;
- ensembl_version: Versão de compilação da anotação Ensembl.
- reference_build: Versão de montagem de referência humana NCBI ou GRC.
- variant_summary: Descrição curta da variante; (Explicação do gene e do cancêr);
- variant_origin: Como a variante começou. (Somatic - Essas variantes surgem pontualmente em algumas células do corpo e não são passadas de pai para filho, causa mais comum de câncer e resultam de danos acumulados no DNA ao longo da vida, Rare Germline- Tipo menos comum de cancêr, está presente nas células germinativas(gametas:espermatozoides e óvulos) e é passada diretamente de pai pra fiho no momento da concepção);
- variant_types: Os tipos de variantes são usados para classificar variantes por termos de Ontologia de Sequência. (missense_variant - Uma variante de sequência, que muda uma ou mais bases, resultando em uma sequência de aminoácidos diferente, mas onde o comprimento é preservado, stop_gained - Uma variante de sequência em que pelo menos uma base de um códon é alterada, resultando em um códon de parada prematuro, levando a um polipeptídeo encurtado, protein_altering_variant - Uma sequência_variante que é prevista para alterar a proteína codificada na sequência de codificação, frameshift_truncamento - Uma variante de deslocamento de quadro que faz com que o quadro de leitura translacional seja encurtado em relação ao recurso de referência, inframe_deletion - Uma variante não sinônima de quadro que exclui bases da sequência de codificação, inframe_insertion - Uma variante não sinônima do quadro que insere bases na sequência de codificação, splice_acceptor_variant - Uma variante que afeta o local aceitador de splicing de RNA (ou seja, a montante ou a montante/3prime de uma borda de exon), splice_donor_variant - Uma variante que afeta o local receptor do doador de RNA (ou seja, a jusante/5prime de uma borda de exon), gene_variant - Uma variante de sequência em que a estrutura do gene é alterada, transcrição_variante - Uma variante de sequência que altera a estrutura da transcrição, loss_of_function_variant - Uma variante de sequência pela qual o produto do gene diminuiu ou aboliu a função, gain_of_function_variant - Uma variante de sequência pela qual uma função nova ou aprimorada é conferida ao produto gênico);
- hgvs_expressions: Strings HGVS definidas pelo usuário seguindo a nomenclatura HGVS que representam esta Variante no nível de DNA, RNA ou proteína;
- civic_variant_evidence_score: 
- allele_registry_id: Identificador de registro de alelo (id de CA) vinculado à página de registro de alelo ClinGen correspondente. Este link é gerado automaticamente usando as coordenadas primárias selecionadas (cromossomo, início, parada, base de referência, base variante).
- clinvar_ids: ID ClinVar definido pelo usuário referenciando esta variante que será vinculada diretamente ao ClinVar. Um valor de “Nenhum especificado” indica que a variante não foi avaliada para uma ID ClinVar. Um valor de “None Found” indica que foi feita uma tentativa de encontrar uma entrada ClinVar correspondente, mas não existe nenhuma. Um valor de “N/A” indica que um registro ClinVar não é aplicável à variante
- variant_aliases: Nomes alternativos para esta variante. Podem ser versões mais detalhadas (por exemplo, 'Val600Glu'), IDs dbSNP ou nomenclatura alternativa usada na literatura.
- description: Descrição do gene e cancêr que será tratado.

## Valores nulos em Civic
- disease                           23
- doid                              33
- phenotypes                      3408
- drugs                           1457
- evidence_direction               179
- clinical_significance            222
- drug_interaction_type           3300
- evidence_level, evidence_statement, citation_id, source_type,citation, evidence_status, evidence_id                         22                
- rating                            90
- chromosome                       457
- start, stop                      458
- reference_bases                 2022
- variant_bases                   2077
- representative_transcript        488
- ensembl_version                  497
- reference_build                  448
- variant_summary                 2532
- variant_origin                   370
- variant_types                    632
- hgvs_expressions, allele_registry_id                 1962
- clinvar_ids                     1839
- variant_aliases                 1505
- description                      762

# Valores duplicados em Civic

- gene, entrez_id, gene_id 3587
- variant 2735
- disease 3681
- doid 3685
- phenotypes 3755
- drugs 3311
- evidence_type 3913
- evidence_direction 3916
- clinical_significance 3903
- variant_id 2386
- is_flagged 3917
- drug_interaction_type 3915
- evidence_level 3913
- evidence_statement 243
- citation_id 2213
- source_type 3916
- citation 2223
- rating 3913
- evidence_status 3917
- evidence_id 21
- chromosome 3895
- start 2935
- stop 2930
- reference_bases 3864
- variant_bases 3876
- representative_transcript 3585
- ensembl_version 3915
- reference_build 3916
- variant_summary 3679
- variant_origin 3914
- variant_types 3823
- hgvs_expressions 3169
- civic_variant_evidence_score 3743
- allele_registry_id 3167
- clinvar_ids 3439
- variant_aliases 3031
- description 3857

# Bibliografia:
**https://docs.civicdb.org/en/latest/api.html#v1-api-deprecated**
https://civic.readthedocs.io/en/latest/model/variants/overview.html
https://civic.readthedocs.io/en/latest/model/assertions/overview.html
https://civic.readthedocs.io/en/latest/model/evidence.html
https://civic.readthedocs.io/en/latest/model/genes.html
**https://civic.readthedocs.io/en/latest/model/evidence/type.html#evidence-type**
https://civic.readthedocs.io/en/latest/model/evidence/significance.html#evidence-significance